In [1]:
from tqdm import tqdm

from utils.elt_config import (
    DataDirectory,
    create_all_schemas,
    get_all_schemas,
    get_cnxn,
    get_info_schema_df,
)

In [2]:
cnxn = get_cnxn(database=":memory:")
create_all_schemas(cnxn)

In [3]:
data_dir = DataDirectory()
data_dir.queries.load_landing_data(cnxn)

In [42]:
info_schema_df = get_info_schema_df(cnxn)
schemas = set(get_all_schemas(info_schema_df))
if "main" not in schemas:
    schemas.add("main")
if "agg" not in schemas:
    schemas.add("agg")
print("schemas:", schemas)
print("columns:", info_schema_df.columns)
info_schema_df.loc[:, ["schema", "name", "column_names"]].head(20)

schemas: {'landing', 'main', 'agg'}
columns: Index(['database', 'schema', 'name', 'column_names', 'column_types',
       'temporary'],
      dtype='object')


,schema,name,column_names
0,landing,assessments,"[assessment_id, module_code, presentation_code..."
1,landing,courses,"[module_code, presentation_code, module_presen..."
2,landing,student_assessment,"[student_id, assessment_id, date_submitted, is..."
3,landing,student_info,"[student_id, module_code, presentation_code, i..."
4,landing,student_registration,"[module_code, presentation_code, student_id, d..."
5,landing,student_vle,"[site_id, student_id, module_code, presentatio..."
6,landing,vle,"[site_id, module_code, presentation_code, acti..."
7,main,activity_type,"[id, activity_type]"
8,main,age_band,"[id, age_band]"
9,main,assessment,"[assessment_id, course_id, module_id, presenta..."


In [45]:
queries = data_dir.queries

for sql_file in tqdm(
    sorted(
        [
            x
            for x in queries.allchildfiles
            if x.path.suffix == ".sql" and "00_landing" not in x.path.parts
        ],
        key=lambda x: str(x.path.parent),
    )
):
    sql = sql_file.path.read_text()
    upper = sql.upper()
    for cmd in ["CREATE", "DROP", "INTO"]:
        if cmd in upper:
            raise ValueError(f"{cmd} in SQL {sql_file.path}")
    stem_parts = sql_file.path.stem.split("_")
    if len(stem_parts) < 2:
        print(sql_file.path.parts[-3:])
        raise ValueError(f"stem {sql_file.path.stem} has less than 2 parts")
    schema = stem_parts[0]
    if schema not in schemas:
        print(sql_file.path.parts[-3:])
        raise ValueError(f"schema {schema} not in schemas {schemas}")
    table = "_".join(stem_parts[1:])
    obj_type = "VIEW" if table.startswith("v_") else "TABLE"
    create_sql = f"CREATE {obj_type} IF NOT EXISTS {schema}.{table} AS {sql}"
    try:
        cnxn.execute(create_sql)
    except Exception as e:
        print(sql_file.path.parts[-3:])
        raise e from e

 40%|████      | 37/92 [00:00<00:00, 193.78it/s]

('02_agg', '01_00_derivative', 'mom_assessments_12.sql')


ValueError: schema mom not in schemas {'landing', 'main', 'agg'}